In [1]:
#Imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
import lxml
import re
import time
import numpy as np
from pprint import pprint

In [2]:
#getting urls setup


url1 = "https://en.wikipedia.org/wiki/List_of_Universal_Pictures_films_(2010%E2%80%932019)"
url2 = "https://en.wikipedia.org/wiki/List_of_Universal_Pictures_films_(2020%E2%80%932029)"
headers = {"User-Agent": "For class web scraping assignment, email: cnelson1845@gmail.com"}
r1 = requests.get(url1, headers=headers)
#abiding by robots.txt to wait 5 seconds between requests
time.sleep(5)
r2 = requests.get(url2, headers=headers)
soup1 = BeautifulSoup(r1.text)
soup2 = BeautifulSoup(r2.text)
print(r1.status_code)
print(r2.status_code)

200
200


In [ ]:
#using pandas to get all the tables and then turning the relevant ones into dataframes
#Doing 2024-2014 results in (after cleaning) a little over 200 entries. All time frames for runtime are given with this set in mind.
#This cell can be changed to expand the time frame
tables1 = pd.read_html(r1.text)
tables2 = pd.read_html(r2.text)
#getting the relevant tables (2014-2019 and 2020-2024)
tables01 = tables1[5:10]
tables02 = tables2[0]
#tables02 includes some 2025 movies, so we will have to remove those later
dfs = []
for table in tables01:
    dfs.append(table)
dfs.append(tables02)
print(dfs)

C:\Users\cnels\AppData\Local\Temp\ipykernel_34708\2640283519.py:2: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables1 = pd.read_html(r1.text)
C:\Users\cnels\AppData\Local\Temp\ipykernel_34708\2640283519.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables2 = pd.read_html(r2.text)


[          Release date                              Title  \
0     January 17, 2014                         Ride Along   
1    February 14, 2014                       Endless Love   
2    February 25, 2014                          Blackfish   
3    February 28, 2014                           Non-Stop   
4          May 9, 2014                          Neighbors   
5         May 30, 2014  A Million Ways to Die in the West   
6         July 4, 2014                        The Anomaly   
7        July 11, 2014                            Boyhood   
8        July 18, 2014                 The Purge: Anarchy   
9        July 25, 2014                               Lucy   
10      August 1, 2014                          Get On Up   
11      August 8, 2014                    Step Up: All In   
12     August 29, 2014                 As Above, So Below   
13  September 19, 2014        A Walk Among the Tombstones   
14  September 26, 2014                   The Boxtrolls[a]   
15    October 10, 2014 

In [4]:
#combining the dataframes
movies = pd.concat(dfs, ignore_index=True)
print(movies.head())

        Release date         Title  \
0   January 17, 2014    Ride Along   
1  February 14, 2014  Endless Love   
2  February 25, 2014     Blackfish   
3  February 28, 2014      Non-Stop   
4        May 9, 2014     Neighbors   

                                               Notes  
0  co-production with Relativity Media, Cube Visi...  
1  co-production with Bluegrass Films and Fake Em...  
2  distribution in Latin America, Scandinavia, Ce...  
3  North American distribution only; produced by ...  
4  distribution only; produced by Point Grey Pict...  


In [5]:
#removing the unneeded notes column
movies = movies.drop('Notes', axis=1)
print(movies.head())

        Release date         Title
0   January 17, 2014    Ride Along
1  February 14, 2014  Endless Love
2  February 25, 2014     Blackfish
3  February 28, 2014      Non-Stop
4        May 9, 2014     Neighbors


In [6]:
#Getting the title and release year from each column and putting it in a dictionary
the_movies = {}
for idx, movie in movies.iterrows():
    #getting title
    title = movie["Title"]
    #getting year
    year = movie["Release date"][-4:]
    #placing into dictionary
    the_movies[title] = year
print(len(the_movies))
pprint(the_movies)

257
{"A Dog's Purpose[N 1]": '2017',
 'A Million Ways to Die in the West': '2014',
 'A Walk Among the Tombstones': '2014',
 'Abigail': '2024',
 'All My Life': '2020',
 'Almost Christmas': '2016',
 'Ambulance': '2022',
 'American Made[N 1]': '2017',
 'Argylle': '2024',
 'Armageddon Time': '2022',
 'As Above, So Below': '2014',
 'Babes': '2024',
 'Beast': '2022',
 'Belfast': '2021',
 'Black Phone 2': '2025',
 'Blackfish': '2014',
 'Blackhat': '2015',
 'Blockers': '2018',
 'Book Club: The Next Chapter': '2023',
 'Boyhood': '2014',
 'Breaking In': '2018',
 'Brian and Charles': '2022',
 "Bridget Jones's Baby": '2016',
 'Bridget Jones: Mad About the Boy §': '2025',
 'Bros': '2022',
 'By the Sea': '2015',
 'Candyman': '2021',
 'Central Intelligence': '2016',
 'Cocaine Bear': '2023',
 'Crimson Peak': '2015',
 'Cyrano': '2022',
 'Daddy Cool': '2017',
 'Dear Evan Hansen': '2021',
 'Death of a Unicorn': '2025',
 'Despicable Me 3': '2017',
 'Despicable Me 4': '2024',
 'Dog Man': '2025',
 'Dolittle

In [7]:
#removing movies from 2025
bad_keys = []
for key,value in the_movies.items():
    if int(value) == 2025:
        bad_keys.append(key)
for key in bad_keys:
    del the_movies[key]
print(len(the_movies))
pprint(the_movies)

232
{"A Dog's Purpose[N 1]": '2017',
 'A Million Ways to Die in the West': '2014',
 'A Walk Among the Tombstones': '2014',
 'Abigail': '2024',
 'All My Life': '2020',
 'Almost Christmas': '2016',
 'Ambulance': '2022',
 'American Made[N 1]': '2017',
 'Argylle': '2024',
 'Armageddon Time': '2022',
 'As Above, So Below': '2014',
 'Babes': '2024',
 'Beast': '2022',
 'Belfast': '2021',
 'Blackfish': '2014',
 'Blackhat': '2015',
 'Blockers': '2018',
 'Book Club: The Next Chapter': '2023',
 'Boyhood': '2014',
 'Breaking In': '2018',
 'Brian and Charles': '2022',
 "Bridget Jones's Baby": '2016',
 'Bros': '2022',
 'By the Sea': '2015',
 'Candyman': '2021',
 'Central Intelligence': '2016',
 'Cocaine Bear': '2023',
 'Crimson Peak': '2015',
 'Cyrano': '2022',
 'Daddy Cool': '2017',
 'Dear Evan Hansen': '2021',
 'Despicable Me 3': '2017',
 'Despicable Me 4': '2024',
 'Dolittle': '2020',
 'Dracula Untold': '2014',
 'Dumb and Dumber To[N 1]': '2014',
 'Easter Sunday[N 1]': '2022',
 'Eileen': '2023',


In [8]:
#formatting titles for The Numbers url and getting all The Numbers urls
#Example 1: https://www.the-numbers.com/movie/As-Above-So-Below#tab=summary
#Example 2: https://www.the-numbers.com/movie/Little-Mermaid-The-(2023)#tab=summary
#Example 3: https://www.the-numbers.com/movie/Terminator-3-Rise-of-the-Machines#tab=summary
#Example 4: https://www.the-numbers.com/movie/Lion-King-1-1-2-The#tab=summary
#Example 5: https://www.the-numbers.com/movie/Spirited-Away-(2001-Japan)#tab=summary
#Example 6: https://www.the-numbers.com/movie/As-Above-So-Below#tab=summary
#Example 7: https://www.the-numbers.com/movie/Airplane#tab=summary
#Example 8: https://www.the-numbers.com/movie/Lilo-and-Stitch-(2025)#tab=summary
urls = []
clean_dict = {}
base = "https://www.the-numbers.com/movie/"
suffix = "#tab=summary"
for title in the_movies:
    clean = title
    #removing specific characters
    clean = clean.replace(" ‡","").replace(" †","").replace(" §","").replace(" *","")
    #removing special characters
    clean = clean.replace(":","").replace("!","").replace("'","").replace(",","").replace(".","").replace("[N 1]","").replace("[a]","")
    #replacing special characters
    clean = clean.replace("/","-").replace("&","and").replace("í","i").replace("á","a")
    #replacing spaces with dashes
    clean = clean.replace(" ","-")
    #moving 'The' and 'A' and 'La' to the end of title 
    clean = re.sub(r'^The-(.+)$', r'\1-The', clean)
    clean = re.sub(r'^A-(.+)$', r'\1-A', clean)
    clean = re.sub(r'^La-(.+)$', r'\1-La', clean)
    #adding it to another dictionary to keep track of years
    clean_dict[clean] = the_movies[title]
    #creating full url with it (make sure year is still there)
    full_url = f"{base}{clean}{suffix}"
    urls.append(full_url)
print(len(urls))
pprint(urls)

232
['https://www.the-numbers.com/movie/Ride-Along#tab=summary',
 'https://www.the-numbers.com/movie/Endless-Love#tab=summary',
 'https://www.the-numbers.com/movie/Blackfish#tab=summary',
 'https://www.the-numbers.com/movie/Non-Stop#tab=summary',
 'https://www.the-numbers.com/movie/Neighbors#tab=summary',
 'https://www.the-numbers.com/movie/Million-Ways-to-Die-in-the-West-A#tab=summary',
 'https://www.the-numbers.com/movie/Anomaly-The#tab=summary',
 'https://www.the-numbers.com/movie/Boyhood#tab=summary',
 'https://www.the-numbers.com/movie/Purge-Anarchy-The#tab=summary',
 'https://www.the-numbers.com/movie/Lucy#tab=summary',
 'https://www.the-numbers.com/movie/Get-On-Up#tab=summary',
 'https://www.the-numbers.com/movie/Step-Up-All-In#tab=summary',
 'https://www.the-numbers.com/movie/As-Above-So-Below#tab=summary',
 'https://www.the-numbers.com/movie/Walk-Among-the-Tombstones-A#tab=summary',
 'https://www.the-numbers.com/movie/Boxtrolls-The#tab=summary',
 'https://www.the-numbers.com/m

In [9]:
#we need to divide up our urls so we don't overload the website, thus we complete this process separately
chunk_size = 50
n_chunks = (len(urls) + chunk_size - 1) // chunk_size   # ceiling division

sections = np.array_split(urls, n_chunks)
print(len(sections))
print(len(sections[0]))
print(sections[0])

5
47
['https://www.the-numbers.com/movie/Ride-Along#tab=summary'
 'https://www.the-numbers.com/movie/Endless-Love#tab=summary'
 'https://www.the-numbers.com/movie/Blackfish#tab=summary'
 'https://www.the-numbers.com/movie/Non-Stop#tab=summary'
 'https://www.the-numbers.com/movie/Neighbors#tab=summary'
 'https://www.the-numbers.com/movie/Million-Ways-to-Die-in-the-West-A#tab=summary'
 'https://www.the-numbers.com/movie/Anomaly-The#tab=summary'
 'https://www.the-numbers.com/movie/Boyhood#tab=summary'
 'https://www.the-numbers.com/movie/Purge-Anarchy-The#tab=summary'
 'https://www.the-numbers.com/movie/Lucy#tab=summary'
 'https://www.the-numbers.com/movie/Get-On-Up#tab=summary'
 'https://www.the-numbers.com/movie/Step-Up-All-In#tab=summary'
 'https://www.the-numbers.com/movie/As-Above-So-Below#tab=summary'
 'https://www.the-numbers.com/movie/Walk-Among-the-Tombstones-A#tab=summary'
 'https://www.the-numbers.com/movie/Boxtrolls-The#tab=summary'
 'https://www.the-numbers.com/movie/Dracula-U

In [10]:
#This takes about 50 minutes.
#fixing the bad ones
#setting up our variables
marked = {}
i = -1
#Just for me to see times during this
start = time.time()
for k,section in enumerate(sections):
    for url in section:
        i += 1
        #abiding by robots.txt
        time.sleep(5)
        r = requests.get(url,headers=headers)
        if r.status_code == 200:
            continue
        #checking if the website broke
        if r.status_code == 503:
            print(f"currently on i: {i} out of 232, with url: {url}")
            break
        current = url
        #placing year at the end of the title
        title = url.rsplit("/", 1)[-1].split("#", 1)[0]
        year = int(clean_dict[title])
        current = re.sub(r'(?=#)', f'-({year})', url)
        time.sleep(5)
        s = requests.get(current,headers=headers)
        if s.status_code == 200:
            urls[i] = current
            continue
        #attempting to add country to make url work
        current = re.sub(r'(?=#)', f'-({year}-United-Kingdom)', url)
        time.sleep(5)
        s = requests.get(current,headers=headers)
        if s.status_code == 200:
            urls[i] = current
            continue
        current = re.sub(r'(?=#)', f'-(UK)-({year})', url)
        time.sleep(5)
        s = requests.get(current,headers=headers)
        if s.status_code == 200:
            urls[i] = current
            continue
        #setting year back by 1
        current = re.sub(r'(?=#)', f'-({year-1})', url)
        time.sleep(5)
        s = requests.get(current,headers=headers)
        if s.status_code == 200:
            urls[i] = current
            continue
        current = re.sub(r'(?=#)', f'-({year}-Japan)', url)
        time.sleep(5)
        s = requests.get(current,headers=headers)
        if s.status_code == 200:
            urls[i] = current
            continue
        current = re.sub(r'(?=#)', f'-(Italy)-({year})', url)
        time.sleep(5)
        s = requests.get(current,headers=headers)
        if s.status_code == 200:
            urls[i] = current
            continue
        current = re.sub(r'(?=#)', f'-({year}-Australia)', url)
        time.sleep(5)
        s = requests.get(current,headers=headers)
        if s.status_code == 200:
            urls[i] = current
            continue
        current = re.sub(r'(?=#)', f'-({year}-France)', url)
        time.sleep(5)
        s = requests.get(current,headers=headers)
        if s.status_code == 200:
            urls[i] = current
            continue
        #if it doesn't we add it to marked
        marked[i] = url
    #Getting time so I know its working
    print(f"Time of section {k+1}/{len(sections)} completion: ~{(time.time()-start)//60} minutes")
#pretty printing it so I can read it    
pprint(marked)

Time of section 1/5 completion: ~7.0 minutes
Time of section 2/5 completion: ~19.0 minutes
Time of section 3/5 completion: ~30.0 minutes
Time of section 4/5 completion: ~40.0 minutes
Time of section 5/5 completion: ~51.0 minutes
{34: np.str_('https://www.the-numbers.com/movie/Tag#tab=summary'),
 50: np.str_('https://www.the-numbers.com/movie/KikoRiki-Legend-of-the-Golden-Dragon#tab=summary'),
 59: np.str_('https://www.the-numbers.com/movie/Purge-Election-Year-The#tab=summary'),
 72: np.str_('https://www.the-numbers.com/movie/Snow-Queen-3-Fire-and-Ice-The#tab=summary'),
 81: np.str_('https://www.the-numbers.com/movie/Colle-La#tab=summary'),
 87: np.str_('https://www.the-numbers.com/movie/Yo-kai-Watch-The-Movie#tab=summary'),
 95: np.str_('https://www.the-numbers.com/movie/I-Am-Losing-Weight#tab=summary'),
 99: np.str_('https://www.the-numbers.com/movie/Loro#tab=summary'),
 110: np.str_('https://www.the-numbers.com/movie/Grinch-The#tab=summary'),
 114: np.str_('https://www.the-numbers.co

In [11]:
#DO NOT ACCIDENTALLY RUN THIS BLOCK AGAIN AFTER RUNNING IT THE FIRST TIME UNLESS YOU ARE RUNNING ALL PREVIOUS BLOCKS AS WELL

#Removing the bad ones, since this means The-Numbers doesn't have them, or the url is not in english, 
#or there was an alternative name that the wikipedia page didn't list, but that was only 1 in this trial
previous = len(urls)
#removing backwards to ensure everything is deleted properly
for key in sorted(marked.keys(), reverse=True):
    urls.pop(key)
#checking again
pprint(urls)
print(len(urls))
print(f"Number of urls removed: {previous - len(urls)}")

['https://www.the-numbers.com/movie/Ride-Along#tab=summary',
 'https://www.the-numbers.com/movie/Endless-Love#tab=summary',
 'https://www.the-numbers.com/movie/Blackfish#tab=summary',
 'https://www.the-numbers.com/movie/Non-Stop#tab=summary',
 'https://www.the-numbers.com/movie/Neighbors#tab=summary',
 'https://www.the-numbers.com/movie/Million-Ways-to-Die-in-the-West-A#tab=summary',
 'https://www.the-numbers.com/movie/Anomaly-The#tab=summary',
 'https://www.the-numbers.com/movie/Boyhood#tab=summary',
 'https://www.the-numbers.com/movie/Purge-Anarchy-The#tab=summary',
 'https://www.the-numbers.com/movie/Lucy#tab=summary',
 'https://www.the-numbers.com/movie/Get-On-Up#tab=summary',
 'https://www.the-numbers.com/movie/Step-Up-All-In#tab=summary',
 'https://www.the-numbers.com/movie/As-Above-So-Below#tab=summary',
 'https://www.the-numbers.com/movie/Walk-Among-the-Tombstones-A#tab=summary',
 'https://www.the-numbers.com/movie/Boxtrolls-The#tab=summary',
 'https://www.the-numbers.com/movie

In [14]:
#This block takes 25-45 minutes
#getting relevant information from each url
title = []
date = []
vid_date = []
inf_adj_dom_rev = []
dom_rev = []
inter_rev = []
total_theater_rev = []
dom_dvd_rev = []
dom_bluray_rev = []
dom_video_rev = []
opening_weekend = []
budget = []
theaters = []
film_rating = []
runtime = []
genre = []
franchise = []
production_method = []

for j,url in enumerate(urls):
    time.sleep(5)
    r = requests.get(url,headers=headers)
    #Checking if something in my cleaning went wrong
    if r.status_code != 200:
        #this means something is wrong with the website, and we will simply consider it as the-numbers not having it.
        print(f"Because status code is not 200-Removed data for url: {url}, with status: {r.status_code}")
        continue
    soup = BeautifulSoup(r.text)
    #getting title and release year
    header = soup.find("h1")
    #This is here just in case
    if header is None:
        #this means something is wrong with the website, and we will simply consider it as the-numbers not having it.
        print(f"Because header is None-Removed data for url: {url}, with header: {header.text}")
        continue
    if re.match(r'^(.*)\s\((\d{4})\)$', header.text) is None:
        #this means something is wrong with the website, and we will simply consider it as the-numbers not having it.
        print(f"Because header doesn't match-Removed data for url: {url}, with header: {header.text}")
        continue
    title_ = re.match(r'^(.*)\s\((\d{4})\)$', header.text).group(1)
    title.append(title_)
    #getting box office numbers
    label = soup.find("b", string="Domestic Box Office")
    if label is not None:
        value_cell = label.find_parent("td").find_next_sibling("td")
        dbo = value_cell.get_text(strip=True)
        dom_rev.append(dbo)
    else:
        dom_rev.append(None)
    label = soup.find("b", string="International Box Office")
    if label is not None:
        value_cell = label.find_parent("td").find_next_sibling("td")
        ibo = value_cell.get_text(strip=True)
        inter_rev.append(ibo)
    else:
        inter_rev.append(None)
    label = soup.find("b", string="Worldwide Box Office")
    if label is not None:
        value_cell = label.find_parent("td").find_next_sibling("td")
        wwbo = value_cell.get_text(strip=True)
        total_theater_rev.append(wwbo)
    else:
        total_theater_rev.append(None)
    label = soup.find("b", string="Infl. Adj. Dom. BO")
    if label is not None:
        value_cell = label.find_parent("td").find_next_sibling("td")
        iadbo = value_cell.get_text(strip=True)
        inf_adj_dom_rev.append(iadbo)
    else:
        inf_adj_dom_rev.append(None)
    #getting video sales
    label = soup.find("b", string="Est. Domestic DVD Sales")
    if label is not None:
        value_cell = label.find_parent("td").find_next_sibling("td")
        dvd = value_cell.get_text(strip=True)
        dom_dvd_rev.append(dvd)
    else:
        dom_dvd_rev.append(None)
    label = soup.find("b", string="Est. Domestic Blu-ray Sales")
    if label is not None:
        value_cell = label.find_parent("td").find_next_sibling("td")
        bluray = value_cell.get_text(strip=True)
        dom_bluray_rev.append(bluray)
    else:
        dom_bluray_rev.append(None)
    label = soup.find("b", string="Total Est. Domestic Video Sales")
    if label is not None:
        value_cell = label.find_parent("td").find_next_sibling("td")
        dvs = value_cell.get_text(strip=True)
        dom_video_rev.append(dvs)
    else:
        dom_video_rev.append(None)
    #getting opening weekend
    label = soup.find("b", string=lambda t: t and "Opening" in t and "Weekend" in t)
    if label is not None:
        value_cell = label.find_parent("td").find_next_sibling("td")
        ow = value_cell.get_text(strip=True)
        opening_weekend.append(ow)
    else:
        opening_weekend.append(None)
    #getting production budget
    label = soup.find("b", string=lambda t: t and "Production" in t and "Budget" in t)
    if label is not None:
        value_cell = label.find_parent("td").find_next_sibling("td")
        pb = value_cell.get_text(strip=True)
        budget.append(pb)
    else:
        budget.append(None)
    #getting theater counts
    label = soup.find("b", string="Theater counts:")
    if label is not None:
        value_cell = label.find_parent("td").find_next_sibling("td")
        tc = value_cell.get_text(strip=True)
        theaters.append(tc)
    else:
        theaters.append(None)
    #getting dates
    label = soup.find("b", string="Domestic Releases:")
    if label is not None:
        value_cell = label.find_parent("td").find_next_sibling("td")
        dom_rel = value_cell.get_text(strip=True)
        date.append(dom_rel)
    else:
        date.append(None)
    label = soup.find("b", string=lambda t: t and "Video" in t and "Release" in t)
    if label is not None:
        value_cell = label.find_parent("td").find_next_sibling("td")
        v_rel = value_cell.get_text(strip=True)
        vid_date.append(v_rel)
    else:
        vid_date.append(None)
    #getting other features of the movie
    label = soup.find("b", string=lambda t: t and "MPAA" in t and "Rating" in t)
    if label is not None:
        value_cell = label.find_parent("td").find_next_sibling("td")
        rating = value_cell.get_text(strip=True)
        film_rating.append(rating)
    else:
        film_rating.append(None)
    label = soup.find("b", string="Running Time:")
    if label is not None:
        value_cell = label.find_parent("td").find_next_sibling("td")
        r_time = value_cell.get_text(strip=True)
        runtime.append(r_time)
    else:
        runtime.append(None)
    label = soup.find("b", string="Franchise:")
    if label is not None:
        value_cell = label.find_parent("td").find_next_sibling("td")
        f = value_cell.get_text(strip=True)
        franchise.append(f)
    else:
        franchise.append(None)
    label = soup.find("b", string="Genre:")
    if label is not None:
        value_cell = label.find_parent("td").find_next_sibling("td")
        g = value_cell.get_text(strip=True)
        genre.append(g)
    else:
        genre.append(None)
    label = soup.find("b", string=lambda t: t and "Method" in t and "Production" in t)
    if label is not None:
        value_cell = label.find_parent("td").find_next_sibling("td")
        pm = value_cell.get_text(strip=True)
        production_method.append(pm)
    else:
        production_method.append(None)
    #Just a progress bar for my sake
    print(f"Finished url {j+1}/{len(urls)}, or {round((j + 1) / len(urls) * 100, 2)}%")
    print(len(title))


Finished url 1/214, or 0.47%
Finished url 2/214, or 0.93%
Finished url 3/214, or 1.4%
Finished url 4/214, or 1.87%
Finished url 5/214, or 2.34%
Finished url 6/214, or 2.8%
Finished url 7/214, or 3.27%
Finished url 8/214, or 3.74%
Finished url 9/214, or 4.21%
Finished url 10/214, or 4.67%
Finished url 11/214, or 5.14%
Finished url 12/214, or 5.61%
Finished url 13/214, or 6.07%
Finished url 14/214, or 6.54%
Finished url 15/214, or 7.01%
Finished url 16/214, or 7.48%
Finished url 17/214, or 7.94%
Finished url 18/214, or 8.41%
Finished url 19/214, or 8.88%
Finished url 20/214, or 9.35%
Finished url 21/214, or 9.81%
Finished url 22/214, or 10.28%
Finished url 23/214, or 10.75%
Finished url 24/214, or 11.21%
Finished url 25/214, or 11.68%
Finished url 26/214, or 12.15%
Finished url 27/214, or 12.62%
Finished url 28/214, or 13.08%
Finished url 29/214, or 13.55%
Finished url 30/214, or 14.02%
Finished url 31/214, or 14.49%
Finished url 32/214, or 14.95%
Finished url 33/214, or 15.42%
Finished 

In [15]:
#putting the information into a dataframe
df = pd.DataFrame({"Title":title,"Release Date":date,"Inflation Adjusted Domestic Revenue":inf_adj_dom_rev,"Domestic Revenue":dom_rev,"International Revenue":inter_rev,
                    "Total Box Office Revenue":total_theater_rev,"Domestic Video Revenue":dom_video_rev,"Opening Weekend":opening_weekend,"Production Budget":budget,
                    "Theater Number":theaters,"MPAA Rating":film_rating,"Runtime":runtime,"Genre":genre,"Franchise":franchise,"Production Method":production_method,
                    "Video Release Date":vid_date,"Domestic DVD Revenue":dom_dvd_rev,"Domestic Bluray Revenue":dom_bluray_rev})
#Looking at the dataframe for cleaning
pd.set_option('display.max_columns', None)
display(df.head())

,Title,Release Date,Inflation Adjusted Domestic Revenue,Domestic Revenue,International Revenue,Total Box Office Revenue,Domestic Video Revenue,Opening Weekend,Production Budget,Theater Number,MPAA Rating,Runtime,Genre,Franchise,Production Method,Video Release Date,Domestic DVD Revenue,Domestic Bluray Revenue
0,Ride Along,"January 17th, 2014 (Wide) byUniversal","$185,781,023","$134,202,565","$19,531,235","$153,733,800","$32,235,491","$41,516,170 (30.9% of total gross)","$25,000,000 (worldwide box office is 6.1 times...","2,663 opening theaters/2,867 max. theaters, 7....","PG-13for sequences of violence, sexual content...",100 minutes,Comedy,Ride Along,Live Action,"April 15th, 2014 byUniversal Home Entertainment","$21,354,607","$10,880,884"
1,Endless Love,"July 17th, 1981 (Wide) byUniversal","$134,255,389","$33,000,000",None,None,None,"$4,162,921 (12.6% of total gross)",None,"580 opening theaters/632 max. theaters, 4.5 we...",R,116 minutes,Drama,None,Live Action,"January 13th, 2015 byUniversal Home Entertainment",None,None
2,Blackfish,"July 19th, 2013 (Limited) byMagnolia Pictures","$2,884,649","$2,073,582","$281,293","$2,354,875","$1,885,360","$75,962 (3.7% of total gross)",None,"5 opening theaters/99 max. theaters, 6.6 weeks...",PG-13for mature thematic elements including di...,83 minutes,Documentary,None,Live Action,"November 12th, 2013 byMagnolia Home Entertainment","$1,654,556","$230,804"
3,Non-Stop,"November 10th, 2000 (Limited)","$80,663","$38,442",None,None,None,"$23,402 (60.9% of total gross)",None,"16 opening theaters/16 max. theaters, 1.9 week...",Not Rated,None,None,None,None,None,None,None
4,Neighbors,"December 18th, 1981 (Wide) byColumbia","$120,158,888","$29,916,207",None,None,"$36,651","$6,481,386 (21.7% of total gross)",None,"1,387 opening theaters/1,415 max. theaters, 5....",R,94 minutes,Black Comedy,None,Live Action,"March 12th, 2019 byMill Creek Entertainment",None,"$36,651"


In [16]:
#cleaning the dataframe
#changing all None values to np.nan
df = df.map(lambda x: np.nan if x is None else x)
df = df.map(lambda x: np.nan if x == 'n/a' else x)
#removing extra parentheses (and other extra information)
cols = ["Opening Weekend","Production Budget","Release Date"]
df[cols] = df[cols].apply(lambda s: s.str.replace(r"\(.*?\)", "", regex=True).str.strip())
cols = ["Release Date","Video Release Date"]
df[cols] = df[cols].apply(lambda s: s.str.replace(r"by.*$", "", regex=True, case=False).str.strip())
df["MPAA Rating"] = df["MPAA Rating"].str.extract(r"^(G|PG|PG-13|R|NC-17|M|GP|X|Not Rated|Unrated|NR)", expand=False).str.strip()
df["Runtime"] = df["Runtime"].str.replace(" minutes", "", regex=True).str.strip()
df["Theater Number"] = df["Theater Number"].str.replace(r"^[^/]*/", "", regex=True)
df["Theater Number"] = df["Theater Number"].str.replace(r"max.*$", "", regex=True).str.strip()
#removing dollar signs
cols = ["Inflation Adjusted Domestic Revenue","Domestic Revenue","International Revenue","Total Box Office Revenue","Domestic Video Revenue","Production Budget","Domestic DVD Revenue","Domestic Bluray Revenue","Opening Weekend"]
df[cols] = df[cols].apply(lambda s: s.str.replace(r"\$", "", regex=True).str.strip())
#removing commas
cols = ["Inflation Adjusted Domestic Revenue","Domestic Revenue","International Revenue","Total Box Office Revenue","Domestic Video Revenue","Production Budget","Domestic DVD Revenue","Domestic Bluray Revenue","Opening Weekend","Theater Number"]
df[cols] = df[cols].apply(lambda s: s.str.replace(r",", "", regex=True).str.strip())
#checking
display(df.head())

,Title,Release Date,Inflation Adjusted Domestic Revenue,Domestic Revenue,International Revenue,Total Box Office Revenue,Domestic Video Revenue,Opening Weekend,Production Budget,Theater Number,MPAA Rating,Runtime,Genre,Franchise,Production Method,Video Release Date,Domestic DVD Revenue,Domestic Bluray Revenue
0,Ride Along,"January 17th, 2014",185781023,134202565,19531235,153733800,32235491,41516170,25000000,2867,PG,100,Comedy,Ride Along,Live Action,"April 15th, 2014",21354607,10880884
1,Endless Love,"July 17th, 1981",134255389,33000000,NaN,NaN,NaN,4162921,NaN,632,R,116,Drama,NaN,Live Action,"January 13th, 2015",NaN,NaN
2,Blackfish,"July 19th, 2013",2884649,2073582,281293,2354875,1885360,75962,NaN,99,PG,83,Documentary,NaN,Live Action,"November 12th, 2013",1654556,230804
3,Non-Stop,"November 10th, 2000",80663,38442,NaN,NaN,NaN,23402,NaN,16,Not Rated,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Neighbors,"December 18th, 1981",120158888,29916207,NaN,NaN,36651,6481386,NaN,1415,R,94,Black Comedy,NaN,Live Action,"March 12th, 2019",NaN,36651


In [17]:
#converting release date to datetime object
cols = ["Release Date", "Video Release Date"]
df[cols] = df[cols].apply(pd.to_datetime, errors="coerce")
#converting everything to its proper object type
cols = ["MPAA Rating","Genre","Franchise","Production Method"]
df[cols] = df[cols].astype(str)
cols = ["Inflation Adjusted Domestic Revenue","Domestic Revenue","International Revenue","Total Box Office Revenue","Opening Weekend","Production Budget","Theater Number","Runtime","Domestic DVD Revenue","Domestic Bluray Revenue", "Domestic Video Revenue"]
df[cols] = df[cols].astype("Int64")
#also making sure turning everything to strings didn't mess up the np.nans
df = df.map(lambda x: np.nan if x == 'nan' else x)
#Checking types
print(df.dtypes)

Title                                          object
Release Date                           datetime64[ns]
Inflation Adjusted Domestic Revenue           float64
Domestic Revenue                              float64
International Revenue                         float64
Total Box Office Revenue                      float64
Domestic Video Revenue                        float64
Opening Weekend                               float64
Production Budget                             float64
Theater Number                                float64
MPAA Rating                                    object
Runtime                                       float64
Genre                                          object
Franchise                                      object
Production Method                              object
Video Release Date                     datetime64[ns]
Domestic DVD Revenue                          float64
Domestic Bluray Revenue                       float64
dtype: object


In [19]:
#getting some unique counts to double check everything 
print("Column: Film-Rating")
print(df["MPAA Rating"].unique())
print("-" * 40)
print("Column: Genre")
print(df["Genre"].unique())
print("-" * 40)
print("Column: Production Method")
print(df["Production Method"].unique())
print("-" * 40)

Column: Film-Rating
['PG' 'R' 'Not Rated' nan]
----------------------------------------
Column: Genre
['Comedy' 'Drama' 'Documentary' nan 'Black Comedy' 'Western' 'Action'
 'Thriller/Suspense' 'Horror' 'Adventure' 'Romantic Comedy' 'Musical'
 'Concert/Performance']
----------------------------------------
Column: Production Method
['Live Action' nan 'Stop-Motion Animation' 'Animation/Live Action'
 'Digital Animation']
----------------------------------------


In [ ]:
#Adding additional columns for analysis
#making month and year their own column
df["Month"] = (df["Release Date"]).dt.month
df["Year"] = (df["Release Date"]).dt.year
#adding season column
def get_season(month):
    if month in [12, 1, 2]:
        return "Winter"
    elif month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    elif month in [9, 10, 11]:
        return "Fall"
    else: 
        return np.nan
df["Season"] = df["Month"].apply(get_season)
#adding profit column (Total Box Office Revenue - production budget)
df["Profit"] = df["Total Box Office Revenue"] - df["Production Budget"]

print(df.head())

          Title Release Date  Inflation Adjusted Domestic Revenue  \
0    Ride Along   2014-01-17                          185781023.0   
1  Endless Love   1981-07-17                          134255389.0   
2     Blackfish   2013-07-19                            2884649.0   
3      Non-Stop   2000-11-10                              80663.0   
4     Neighbors   1981-12-18                          120158888.0   

   Domestic Revenue  International Revenue  Total Box Office Revenue  \
0       134202565.0             19531235.0               153733800.0   
1        33000000.0                    NaN                       NaN   
2         2073582.0               281293.0                 2354875.0   
3           38442.0                    NaN                       NaN   
4        29916207.0                    NaN                       NaN   

   Domestic Video Revenue  Opening Weekend  Production Budget  Theater Number  \
0              32235491.0       41516170.0         25000000.0          

In [22]:
#saving the dataframe as a csv
df.to_csv("movie_data.csv", index=False)